#  Forward pass

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы: 
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann 
* https://pytorch.org/docs/stable/generated/torch.matmul.html
* https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/
* https://machinelearningmastery.com/loss-and-loss-functions-for-training-deep-learning-neural-networks/
* https://kidger.site/thoughts/jaxtyping/
* https://github.com/patrick-kidger/torchtyping/tree/master

## Задачи для совместного разбора

In [2]:
pip install torchtyping

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [torchtyping]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
from torchtyping import TensorType, patch_typeguard
from typeguard import typechecked
import torch as th
import torch

Scalar = TensorType[()]
patch_typeguard()

1\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте нейрон с заданными весами `weights` и `bias`. Пропустите вектор `inputs` через нейрон и выведите результат. 

In [17]:
class Neuron:
    def __init__(self, n_features: int, bias: float):
        # <создать атрибуты объекта weights и bias>
        self.weights: TensorType[n_features] =  torch.ones(n_features)
        self.bias: float = bias

    def forward(self, inputs: TensorType["n_features"]) -> Scalar:
        result = th.dot(inputs, self.weights)
        if self.bias is not None:
            result += self.bias
        return result# <реализовать логику нейрона>


In [19]:
import torch as th
inputs = th.tensor([1.0, 2.0, 3.0, 4.0])

In [21]:
neuron = Neuron(4, True)
neuron.forward(inputs)

tensor(11.)

2\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ReLU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/f4353f4e3e484130504049599d2e7b040793e1eb)

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

In [23]:
class ReLU:
    @typechecked
    def forward(self, inputs: TensorType["n_features"]) -> TensorType["n_features"]:
        return th.maximum(inputs, th.tensor(0.0))# <реализовать логику ReLU>


3\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь MSE:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e258221518869aa1c6561bb75b99476c4734108e)
где $Y_i$ - правильный ответ для примера $i$, $\hat{Y_i}$ - предсказание модели для примера $i$, $n$ - количество примеров в батче.

In [25]:
class MSELoss:
    @typechecked
    def forward(self, y_pred: TensorType["batch"], y_true: TensorType["batch"]) -> Scalar:
        return th.mean((y_pred - y_true) **2)# <реализовать логику MSE>

In [27]:
y_pred = th.tensor([1.0, 3.0, 5.0])
y_true = th.tensor([2.0, 3.0, 4.0])

## Задачи для самостоятельного решения

### Cоздание полносвязных слоев

<p class="task" id="1"></p>

1\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте полносвязный слой из `n_neurons` нейронов с `n_features` весами у каждого нейрона (инициализируются из стандартного нормального распределения) и опциональным вектором смещения. 

$$y = xW^T + b$$

Пропустите вектор `inputs` через слой и выведите результат. Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.

- [ ] Проверено на семинаре

In [35]:
class Linear:
    def __init__(self, n_neurons: int, n_features: int, bias: bool = False) -> None:
        self.weights: TensorType[n_features] =  th.randn(n_neurons, n_features)
        self.bias: float = th.randn(n_neurons) if bias else None

    def forward(self, inputs: TensorType["batch", "feats"]) -> TensorType["batch", "n_neurons"]:
        output = th.matmul(inputs, self.weights.T)
        if self.bias is not None:
            output += self.bias
        return output

In [85]:
linear = Linear(4, len(inputs), True)
linear.forward(inputs)

tensor([-1.7634, -5.1476,  4.0991, -0.7604])

<p class="task" id="2"></p>

2\. Используя решение предыдущей задачи, создайте 2 полносвязных слоя и пропустите тензор `inputs` последовательно через эти два слоя. Количество нейронов в первом слое выберите произвольно, количество нейронов во втором слое выберите так, чтобы результатом прогона являлась матрица `batch_size x 7`. 

- [ ] Проверено на семинаре

In [101]:
batch_inputs = inputs.unsqueeze(0)  #TensorType["batch", "feats"]

n_features = 4
n_neurons_1 = 6  
linear1 = Linear(n_neurons_1, n_features, bias=True)


output1 = linear1.forward(batch_inputs)
print(f"выход первого слоя: {output1}")

n_neurons_2 = 7  
linear2 = Linear(n_neurons_2, n_neurons_1, bias=True)


output2 = linear2.forward(output1)
print(f"выход: {output2}")


output2.shape

выход первого слоя: tensor([[ 4.6278, -1.3499, -6.2242,  2.0020, -9.1487, -8.8083]])
выход: tensor([[ -6.9187,   4.3071,  24.2933,  41.8108, -21.3752,  26.5358,  10.0904]])


torch.Size([1, 7])

### Создание функций активации

<p class="task" id="3"></p>

3\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации softmax:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/6d7500d980c313da83e4117da701bf7c8f1982f5)

$$\overrightarrow{x} = (x_1, ..., x_J)$$

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации. Строки матрицы трактовать как выходы линейного слоя некоторого классификатора для 4 различных примеров. Функция должна применяться переданной на вход матрице построчно.

- [ ] Проверено на семинаре

In [176]:
class Softmax:
    def forward(self, inputs: TensorType["batch", "feats"]) -> TensorType["batch", "feats"]:
        exp_inputs = th.exp(inputs-  th.max(inputs, dim=1, keepdim=True).values)
        return exp_inputs / th.sum(exp_inputs, dim=1, keepdim=True)# <реализовать логику Softmax>

softmax = Softmax()
softmax_result = softmax.forward(th.randn(4, 3))
print(softmax_result.size())
print(th.sum(softmax_result, dim=1))

torch.Size([4, 3])
tensor([1.0000, 1.0000, 1.0000, 1.0000])


In [164]:
th.randn(4, 3)

tensor([[ 0.4249,  0.1640,  0.7527],
        [-0.2598,  0.6996, -0.7397],
        [ 0.8849, -0.1195, -0.4032],
        [-0.1703,  0.1313, -0.6358]])

<p class="task" id="4"></p>

4 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ELU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/eb23becd37c3602c4838e53f532163279192e4fd)

Создайте матрицу размера 4x3, заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

- [ ] Проверено на семинаре

In [113]:
class ELU:
    def __init__(self, alpha: float) -> None:
        self.alpha = alpha
    
    def forward(self, inputs: TensorType["batch", "feats"]) -> TensorType["batch", "feats"]:
        return th.where(inputs > 0, inputs, self.alpha * (th.exp(inputs) - 1))


elu = ELU(0.5)
elu_result = elu.forward(th.randn(4, 3))
print(elu_result.size())
print(elu_result)

torch.Size([4, 3])
tensor([[ 0.1083, -0.2983,  1.3293],
        [ 1.5927,  0.6134,  2.1727],
        [-0.1595,  0.1295,  1.3103],
        [ 0.1431, -0.0638,  0.3363]])


### Создание функции потерь

<p class="task" id="5"></p>

5 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь CrossEntropyLoss:

$$y_i = (y_{i,1},...,y_{i,k})$$ 

<img src="https://i.ibb.co/93gy1dN/Screenshot-9.png" width="200">

$$ CrossEntropyLoss = \frac{1}{n}\sum_{i=1}^{n}{L_i}$$
где $y_i$ - вектор правильных ответов для примера $i$, $\hat{y_i}$ - вектор предсказаний модели для примера $i$; $k$ - количество классов, $n$ - количество примеров в батче.

Создайте полносвязный слой с 2 нейронами и прогнать через него батч `inputs`. Полученный результат пропустите через функцию активации Softmax. Посчитайте значение функции потерь, трактуя вектор `y` как вектор правильных ответов.

- [ ] Проверено на семинаре



In [158]:
inputs = th.tensor([
    [1.0, 2.0, 3.0, 4.0],   
    [1.0, 3.0, 2.0, 0.0]    
])  

y = th.tensor([
    [0.0, 1.0], 
    [1.0, 0.0]  
])  


linear = Linear(n_neurons=2, n_features=4, bias=True)
logits = linear.forward(inputs)  

softmax_preds = softmax.forward(logits.unsqueeze(0))  

loss = -th.sum(y * th.log(softmax_preds)) / y.shape[0]

print("Предсказания (после softmax):", softmax_preds)
print("Значение функции потерь:", loss.item())


Предсказания (после softmax): tensor([[[0.9953, 0.9934],
         [0.0047, 0.0066]]])
Значение функции потерь: 2.681881904602051


<p class="task" id="6"></p>

6 Модифицируйте MSE, добавив L2-регуляризацию.

$$MSE_R = MSE + \lambda\sum_{i=1}^{m}w_i^2$$

где $\lambda$ - коэффициент регуляризации; $w_i$ - веса модели.

- [ ] Проверено на семинаре

In [144]:
class MSERegularized:
    def __init__(self, lambda_: float) -> None:
        self.lambda_ = lambda_

    def data_loss(
            self, 
            y_pred: TensorType["batch"], 
            y_true: TensorType["batch"],
    ) -> Scalar:
        return th.mean((y_pred - y_true) **2)# <реализовать логику MSE>

    def reg_loss(self, weights: TensorType["batch", 1])  -> Scalar:
        return th.sum(weights ** 2)


    def forward(
        self, 
        y_pred: TensorType["batch"], 
        y_true: TensorType["batch"], 
        weights: TensorType["batch", 1],
    ) -> Scalar:
        return self.data_loss(y_pred, y_true) + self.lambda_ * self.reg_loss(weights)



mse_reg = MSERegularized(0.01)
weights = th.randn(10, 1)
print(mse_reg.forward(y_pred, y_true, weights))

tensor(0.8063)
